Run all cells for the model to work

In [12]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from transformers import TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType, PeftModel

In [2]:
# bert
BERT_MODEL_NAME = "cl-tohoku/bert-base-japanese"

# ffnn
FFNN_MODEL_PATH = "models/tone_classifier.pt"

# gpt
LORA_MODEL_PATH = "models/rinna-lora-finetuned"
GPT_MODEL_NAME = "rinna/japanese-gpt2-medium"

# dataset files for labels
TONE_TRAIN_FILE_1 = 'data/jchat_paired.csv' 
TONE_TRAIN_FILE_2 = 'data/chigiri_train_w_tone.csv'

In [3]:
# load pre-trained models
# bert (bert tokenizer for FFNN tone prediction)
bert_tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME, use_fast=False)
bert_model = AutoModel.from_pretrained(BERT_MODEL_NAME)

In [4]:
def gpt_tokenize_function(data, max_length=512):
    """
    Tokenize a sentence.
    
    Args:
        data: a row from dataset
        max_length: maximum length of returned result
        
    Return:
        tokenized: tokenized sentence
    """
    # reformat the prompts into a full sentnece
    prompts = [
        instruction + "<NL>" + user + "<NL>" + output + gpt_tokenizer.eos_token
        for instruction, user, output in zip(data["instruction"], data["input"], data["output"])
    ]

    # tokenize using the tokenizer
    tokenized = gpt_tokenizer(prompts, truncation=True, padding="max_length", max_length=max_length)
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

def load_trained_LoRA(base_model, lora_path):
    """
    Load in the trained LoRA layers to the base model.
    
    Args:
       base_model: base gpt model. Should be the same one as LoRA was trained
       lora_path: LORA_MODEL_PATH
       
    Return:
        combined_full_model
    """
    return PeftModel.from_pretrained(base_model, lora_path, is_trainable=True)

In [7]:
# gpt
gpt_tokenizer = AutoTokenizer.from_pretrained(GPT_MODEL_NAME, use_fast=False)
gpt_tokenizer.add_special_tokens({"additional_special_tokens": ["###指示:", "###ユーザー:", "###キャラ:"]})
gpt_base_model = AutoModelForCausalLM.from_pretrained(GPT_MODEL_NAME)
gpt_base_model.resize_token_embeddings(len(gpt_tokenizer))

full_gpt_model = load_trained_LoRA(gpt_base_model, LORA_MODEL_PATH)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [9]:
# FFNN
# get out all tone from both files
labels = set()
for file_path in [TONE_TRAIN_FILE_1, TONE_TRAIN_FILE_2]:
    df_data = pd.read_csv(file_path)
    labels = labels | set(df_data["tone"])

# sort the labels so it will be the same every time
ls_label = sorted(list(labels))
ls_label

['anger',
 'disgust',
 'fear',
 'joy',
 'neutral',
 'sadness',
 'surprise',
 'ツンデレっぽく、素直じゃないけど質問には答える',
 'ネタっぽく・軽いノリでふざけて',
 '冷静に説明・論理的に助言・分析する',
 '前向きで熱く、感情的になっている',
 '心を開きたくないけど、実は話してる',
 '怒っている・挑発的・強く主張する',
 '悔しさと怒りを爆発させる',
 '普通の会話・挨拶っぽく軽い返事・気分が良い',
 '激しい語気で挑発',
 '短く突っ込む・反射的なリアクション',
 '素直じゃない・気分屋・ワガママ・理不尽的',
 '緊張・心の声・ためらい',
 '自信満々に・堂々と・しどろもどろに',
 '自分の話を語る・心情を吐露する',
 '言いたくない・感情を隠す・葛藤をにじませる']

In [10]:
dict_translate_tone = {"anger": " 怒っている",
                      "disgust": "嫌悪感を持っている",
                      "fear": "怯えている",
                      "joy": "嬉しそうな",
                      "neutral": "落ち着いた",
                      "sadness": "悲しそう",
                      "surprise": "驚いている"}

In [13]:
class FFNN(nn.Module):
    def __init__(self, input_dim=768, hidden_units=128, num_classes=len(labels)):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_units)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_units, num_classes)

    def forward(self, X):
        X = self.fc1(X)
        X = self.relu(X)
        X = self.dropout(X)
        return self.fc2(X)

def load_model(path, input_dim=768, hidden_units=128, num_classes=len(labels)):
    """Load model parameters from file and return the model."""
    model = FFNN(input_dim=input_dim, hidden_units=hidden_units, num_classes=num_classes)
    model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    model.eval()
    print(f"FFNN Model loaded")
    return model

ffnn_model = load_model(FFNN_MODEL_PATH)

FFNN Model loaded


# Full function

In [17]:
def predict_tone(text, ffnn_model, bert_tokenizer, bert_embedding, ls_label):
    """
    Use the FFNN model to predict response tone.
    
    Args:
        text: plain user inputted text
        ffnn_model: the trained FFNN model
        bert_tokenizer: loaded Tohoku bert tokenizer
        bert_embedding: loaded Tohoku bert embedding model
        ls_label: a list of labels of tone
        
    Return:
        predicted tone
    """
    with torch.no_grad():
        # tokenize and do embedding
        inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        outputs = bert_embedding(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
        
        # get prediction
        logits = ffnn_model(embedding)
        pred_idx = torch.argmax(logits, dim=1).item()
        
    return ls_label[pred_idx]

In [22]:
def generate_char_response(include_tone=True):
    """
    Full function to generate character's response. Ask the user to give input
    and generate response using the created model.
    
    Args:
        include_tone (bool): whether or not to use the FFNN model to predict responsing tone
    """
    # ask user to input text
    user_input = input()
    
    # step 1: use bert to tokenized, embedding, and use
    if include_tone:
        pred_tone = predict_tone(user_input, ffnn_model, bert_tokenizer, bert_model, ls_label)
        
        # tone in j-chat are in English, change to japanese
        if pred_tone in dict_translate_tone:
            reply_tone = dict_translate_tone[pred_tone]
        else:
            reply_tone = pred_tone
    
        # combint tone to the front of the user input
        user_input = pred_tone + "口調で返事をください。<NL>ユーザー:" + user_input + "<NL>キャラ:"
    else:
        user_input = "ユーザー:" + user_input
        
    # step 2: put into fine_tuned rinna/gpt and generat response
    inputs = gpt_tokenizer(user_input, return_tensors="pt").to(full_gpt_model.device)
    output = full_gpt_model.generate(
        **inputs,
        max_new_tokens=30,
        eos_token_id=gpt_tokenizer.eos_token_id,
        pad_token_id=gpt_tokenizer.pad_token_id,
        repetition_penalty=1.2,
    )

    # step 3: print out the result
    print(gpt_tokenizer.decode(output[0], skip_special_tokens=True))

# Test the performance

In [33]:
# test for response with tone
generate_char_response()

君の武器を教えてくれ。
冷静に説明・論理的に助言・分析する口調で返事をください。<>ユーザー:君の武器を教えてくれ。<>キャラ:##キャラ: 俺はお前が最強だと思ってるからな! ##キャラ: 俺はお前の強さを信じてるんだよ


In [24]:
# test for response without tone
generate_char_response(False)

君の武器を教えてくれ。
ユーザー:君の武器を教えてくれ。##キャラ: 俺は、お前に勝てる気がしないんだよ! ##キャラ: いや、俺もそう


In [42]:
# compair with not-fine-tuned model
input_text = "ユーザー: 君の武器を教えてくれ。キャラ:"

inputs = gpt_tokenizer(input_text, return_tensors="pt").to(gpt_base_model.device)
output = gpt_base_model.generate(
    **inputs,
    max_new_tokens=50,
    eos_token_id=gpt_tokenizer.eos_token_id,
    pad_token_id=gpt_tokenizer.pad_token_id,
    repetition_penalty=1.2,
)
print(gpt_tokenizer.decode(output[0], skip_special_tokens=True))


ユーザー: 君の武器を教えてくれ。キャラ:僕は、このゲームをプレイするために必要なものを全て持っているんだ。 君が望むなら、僕はこのゲームで得た知識と経験を使って、君に教えてあげるよ。 君は、このゲームで得た知識と経験を使って


#### Try for several times

In [35]:
print("With FFNN tone:")
generate_char_response()
print()
print("Without FFNN tone:")
generate_char_response(False)

With FFNN tone:
世界一のストライカー‬になりたいんだ
冷静に説明・論理的に助言・分析する口調で返事をください。<>ユーザー:世界一のストライカーになりたいんだ<>キャラ:4||0||||0||0||0||||4||なが="2"|-||0||0||2||1本が3||0||3||1にする="2"|1||0||=2|-||||5||2||0の町0||0||1||0="2"|-||『""』0||0||||||||||||||||0||1拮

Without FFNN tone:
世界一のストライカー‬になりたいんだ
ユーザー:世界一のストライカーになりたいんだ###キャラ: お前は俺に勝てるのか? ##キャラ: お前が俺を倒せるか! #


In [40]:
print("With FFNN tone:")
generate_char_response()
print()
print("Without FFNN tone:")
generate_char_response(False)

With FFNN tone:
怪我した時、正直どんな気持ちだった？
冷静に説明・論理的に助言・分析する口調で返事をください。<>ユーザー:怪我した時、正直どんな気持ちだった?<>キャラ:###キャラ: 俺はお前の味方だ! ##キャラ: お前がそう思ってくれるならそれでいいんだ

Without FFNN tone:
怪我した時、正直どんな気持ちだった？
ユーザー:怪我した時、正直どんな気持ちだった?4||0|| 当日有権者数:0人/1人 ###キャラ: 俺はお前の味方だ! ##キャラ: お前


In [39]:
print("With FFNN tone:")
generate_char_response()
print()
print("Without FFNN tone:")
generate_char_response(False)

With FFNN tone:
君にとって「ストライカー」って何？
冷静に説明・論理的に助言・分析する口調で返事をください。<>ユーザー:君にとって「ストライカー」って何?<>キャラ:ループしてるの? ###キャラ: 俺はお前がストライカーだと思ってるからな! ##キャラ: い

Without FFNN tone:
君にとって「ストライカー」って何？
ユーザー:君にとって「ストライカー」って何?###キャラ: お前は俺のヒーローだ! ##キャラ: お前が俺を倒せるなら、俺も倒
